In [32]:
import pandas as pd
import numpy as nm
import requests
import time
import re
from bs4 import BeautifulSoup
import os
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import MySQLdb
from sqlalchemy import create_engine
import pymysql

# Part 1:  SHORT, LET GET STARTED


In [33]:
url = 'http://numbersapi.com/0..99/trivia'
headers = {'user-agent': 'Mozilla/5.0'}
response = requests.get(url, headers = headers, )

In [34]:
response_dictionary = response.json() #parses the response into a python object

In [35]:
# print the desired result

for i in range(0,100):
    item = response_dictionary[str(i)]
    print(str(i).zfill(3), " ", "-", " ",item)

000   -   0 is the coldest possible temperature old the Kelvin scale.
001   -   1 is the number of Gods in monotheism.
002   -   2 is the first magic number in physics.
003   -   3 is number of performers in a trio.
004   -   4 is the number of chambers the mammalian heart consists of.
005   -   5 is the most common number of gears for automobiles with manual transmission.
006   -   6 is the number of days taken to create the world by Yahweh in the Old Testament Book of Genesis, and the number of the day on which humankind were created.
007   -   7 is the approximate number of years in the lifespan of Irish Wolfhound dogs.
008   -   8 is the number of legs that arachnids have.
009   -   9 is the number of circles of Hell in Dante's Divine Comedy.
010   -   10 is the Number of dots in a tetractys.
011   -   11 is the number of incarnations of The Doctor in BBC sci-fi series Doctor Who.
012   -   12 is the number of keys in any standard digital telephone (1 through 9, 0, * and #).
013   

# Part 2: Web-scraping: Which sellers advertise/sponsor on eBay

Q2 (a) Take the code your group developed for assignment 2 and fix everything that may not have been optimal in your previous solution. Alternatively, but less cool, you may use our solution code as your starting point. (You will use this for the next part. You do not need to include this code in your submission.)


In [36]:
## In my previous code I could have used functions for doing repetitive tasks. 
## I have tried to implement that in code this time

## How would you modify your search query to only include buy-it-now (non-auction) items?

    # LH_BIN=1
    # 'https://www.ebay.com/sch/i.html?_nkw=iphone+11+pro&LH_BIN=1’

## What is the GET request's variable name corresponding to buy-it-now searches?
    
    # LH_BIN

## How would you modify your search query to include 100 items per search result page? 

    # 'https://www.ebay.com/sch/i.html?_nkw=iphone+11+pro&LH_BIN=1&_ipg=100'

## What is the GET request's variable name corresponding to items per page searches? 
    
    # _ipg


Q2 (b) Modify your previous code to search for buy-it-now listings of <keyword> and limit the number of items to 100 per page. Use this guide to select the right <keyword> assigned to you:
   
   If your last name on Canvas starts with M-R use <keyword>: "iphone 11 pro"
    
Identify all the sponsored items.
For the first 10 pages of 100 items/page, save all the URLs of sponsored items' pages to the file "sponsored.txt"  and all the URLs of non-sponsored items' pages to the file "non-sponsored.txt" in the same directory as your code. (One URL per line in each file)


In [37]:

# url = 'https://www.ebay.com/sch/i.html?_nkw=iphone+11+pro'

# url = 'https://www.ebay.com/sch/i.html?_nkw=iphone+11+pro&LH_BIN=1'

url = 'https://www.ebay.com/sch/i.html?_nkw=iphone+11+pro&LH_BIN=1&_ipg=100'


headers = {'user-agent': 'Mozilla/5.0'}
response = requests.get(url, headers = headers)

In [38]:
url = 'https://www.ebay.com/sch/i.html?_nkw=iphone+11+pro&LH_BIN=1&_ipg=100'


headers = {'user-agent': 'Mozilla/5.0'}
response = requests.get(url, headers = headers)

In [39]:
# Iterates through pages to get the sponsored and non-sponsored links
sponsored_list = []
nonsponsored_list = []



for i in range(1, 11):
    url = 'https://www.ebay.com/sch/i.html?_nkw=iphone+11+pro&LH_BIN=1&_ipg=100&&_pgn=' + str(i)
    headers = {'user-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers = headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    divs = soup.findAll("div",{"class":"s-item__info clearfix"})
    
    
    for div in divs:
        new_div = div.find("div", {"class":"s-item__title--tagblock"})
        sponsored_tag = None
        if new_div is not None:        
            sponsored_tag= re.match(r".*[Ss].*[Pp].*[Oo].*[Nn].*[Ss].*[Oo].*[Rr].*[Ee].*[Dd].*", new_div.text) #Matching sponsored strings
        
        a = div.find("a", href = True)
        

        if sponsored_tag is not None:
            sponsored_list.append(a['href'])
        else:
            nonsponsored_list.append(a['href'])


In [40]:
#saves the sponsored links to a text file

with open("sponsored.txt",'w',encoding = 'utf-8') as f:
    for link in sponsored_list:
        f.write(link + '\n')
        
    

In [41]:
#saves the non-sponsored links to a text file

with open("nonsponsored.txt",'w',encoding = 'utf-8') as f:
    for link in nonsponsored_list:
        f.write(link + '\n')


Q 2 (c) Create two folders in the same directory as your code and name them "sponsored" and "non-sponsored". Write a program that opens the two files in (b) and downloads each of the pages (URLs) into the folders "sponsored" and "non-sponsored". Each file should be named as "<item-id>.htm" where you replace "item-id" with the ID of the item you are saving. E.g., "264616053293.htm" for the item with ID "264616053293". Note it is always good to put a 2-second pause between queries. Make sure to catch an error and continue if your query runs into problems connecting to eBay (e.g., if your internet is down for 5 seconds, you don't want your entire code to crash).

In [42]:
#Creates directory for sponsored and non-sponsored links
if not os.path.exists('sponsored'):
    os.makedirs('sponsored')
if not os.path.exists('nonsponsored'):
    os.makedirs('nonsponsored')

In [43]:
#Function Writes files for sponsored and non-sponsored links in directory

def writefiles(dir_name):
    
    with open(dir_name+".txt", 'r', encoding = 'utf-8') as f:
        for url in f.readlines():
            
            headers = {'user-agent': 'Mozilla/5.0'}

            try:     
                response = requests.get(url, headers = headers)
                soup = BeautifulSoup(response.content, 'html.parser')
                item_id = soup.findAll("div",{"id":"descItemNumber"})[0].text
                filename = dir_name + "/" + item_id + ".htm"
                with open(filename, 'w', encoding = 'utf-8') as file:
                    file.write(response.text)

            except requests.exceptions.RequestException as err:
                print ("Request Exception",err,"URL:", url)
            except requests.exceptions.HTTPError as errh:
                print ("Http Error:",errh,"URL:", url)
            except requests.exceptions.ConnectionError as errc:
                print ("Error Connecting:",errc,"URL:", url)
            except requests.exceptions.Timeout as errt:
                print ("Timeout Error:",errt,"URL:", url)
            except Exception as e:
                print("OOps: Something Else", e, "URL:", url)

            time.sleep(2)


In [44]:
sponsored_files = writefiles("sponsored")

In [45]:
nonsponsored_files = writefiles("nonsponsored")


Q 2 d) Write a separate piece of code that loops through the pages you downloaded in (c) and opens and parses them into a Python or Java xxxxsoup-object. Identify and select:
seller name, seller score, item price, # items sold (what happens when no not present), best offer available, title, returns allowed, shipping price, condition (e.g., used, new, like new, seller refurbished, ...).
In your code, highlight the selector command you choose to obtain each element using comments.


In [49]:
# Function reads the files in the directory and fetches all the information required from each page

def read_files_data(dir_name):
    file_list = os.listdir(dir_name)
    dataframe = pd.DataFrame(columns= ['item_id','seller_name', 'seller_score' , 
                                       'item_price', 'item_sold', 
                                       'best_offer_flag', 'title', 
                                       'returns_allowed', 'condition1', 
                                       'is_sponsored', 'shipping_calc',
                                       'shipping_price'])
    for file in file_list:
        with open(dir_name+ "/" + file, 'r', encoding = 'utf-8') as file:
            content = file.read()

        soup = BeautifulSoup(content, "html")

#Item_id

        item_id = soup.findAll("div",{"id":"descItemNumber"})[0].text
        
# Seller Name --> Div (Class: si-content) --> Span (Class: mbg-nw)  
        seller_name = soup.find('div',{'class':'si-content'})
        seller_name = seller_name.find('span',{'class':'mbg-nw'})
        if seller_name is not None:
            seller_name = seller_name.text

#Seller Score --> Div (Class: si-content) --> Span (Class: mbg-1) --> a 
        seller_score_value = soup.find('div',{'class':'si-content'})
        seller_score_value = seller_score_value.find('span', {'class':'mbg-l'})
        seller_score_value = seller_score_value.find('a')
        seller_score = None
        if seller_score_value is not None:
            seller_score = seller_score_value.text

#Item price --> Span (itemprop: price) or Span (id: prcIsum) or Span (id: mm-saleDscPrc) or Span (id: prcIsum_bidPrice)
        item_price = soup.find('span', {'itemprop':'price'})
        if item_price is None:
            item_price = soup.find('span', {'id':'prcIsum'})
        if item_price is None:
            item_price = soup.find('span', {'id' : 'mm-saleDscPrc'})
        if item_price is None:
            item_price = soup.find('span', {'id' : 'prcIsum_bidPrice'})
        if item_price is not None:
            item_price_value = item_price.text
        if item_price is None:
            item_price_value = None
            

#Item sold --> a (Class: vi-txt-underline)
        item_sold_value = soup.find('a', {'class':'vi-txt-underline'})
        item_sold = None
        if item_sold_value is not None:
            item_sold = item_sold_value.text

#Best Offer Value --> span (id: boBtn_btn_parent)
        best_offer_value = soup.find('span', {'id':'boBtn_btn_parent'})
        best_offer_flag = False
        if best_offer_value is not None:
            best_offer_flag = True

#Title --> h1 (id: itenTitle)
        title = soup.find('h1', {'id':'itemTitle'}).text
        title = bytes(title, 'utf-8').decode('utf-8','ignore')
        
        
#         returns = soup.findAll('span', {'class':'w2b-sgl'})
#         return_val = 0
#         for value in returns:
#             if(value.text == "Returns accepted"):
#                 return_val = 1

# Returns Allowed --> Span(id: vi-ret-accrd-txt)
        returns1 = soup.find('span', {'id': 'vi-ret-accrd-txt'})
        returns_allowed = True
        if returns1 is not None:
            if re.search('Seller does not accept returns',returns1.text):
                returns_allowed = False

                
#Shipping --> span (Class: shSummary)

        shipping = soup.find('span', {'id':'shSummary'})
        shipping_calc = False
 
        if shipping is not None:
            if re.search("free", shipping.text.lower()):
                shipping_price = 0
                shipping_calc = False
            elif re.search('calculate', shipping.text.lower()):
                shipping_price = None
                shipping_calc = True
            elif re.search('\$\d', shipping.text):
                shipping_price = shipping.find('span',{'id':'fshippingCost'}).text

                if re.search('US', shipping_price):                
                    shipping_price = re.findall('\$\d*.?\d*', shipping_price)
                    shipping_price = re.sub('[^0-9]','', str(shipping_price[-1]))
                    
                else:                    
                    shipping_price = re.sub('[^0-9]','', str(shipping_price))
                    

        else:
            shipping_price = None
       
        
        
#Condition --> Dic (Class: condText)          

        condition = soup.find('div', {'class':'condText'})
        if condition is not None:
            condition = condition.text
        if condition is None:
            condition = "Not Given"
            
#Sponsored --> Just check the name of directory
        is_sponsored = 'No'
        if dir_name == 'sponsored':
            is_sponsored = 'Yes'
    
        data = {'item_id' : item_id,
                'seller_name' :seller_name, 'seller_score' : seller_score,
                'item_price' : item_price_value, 'item_sold' : item_sold,
                'best_offer_flag' : best_offer_flag, 'title' : title, 
                'returns_allowed' : returns_allowed,  'condition1' : condition, 
                'is_sponsored' : is_sponsored, 'shipping_calc':shipping_calc,
                 'shipping_price' : shipping_price
               }
        
        
        dataframe = dataframe.append(data, ignore_index=True)
    

    
    
    return dataframe

In [50]:
sponsored_df = read_files_data("sponsored")

In [51]:
nonsponsored_df = read_files_data("nonsponsored")

In [52]:
# Standardizes the values to be stored in the database

def convert_price(price):
    price  = re.sub(r'[a-zA-Z /$,]*', r'', str(price))
    if '.' not in price:
        price += "00"
    price = price.replace('.' , "")
    return int(price)

def sold_items(sold):
    sold = re.sub('[^0-9]','', str(sold))
    if sold == '0' or sold.strip() == '':
        sold = None
    return sold


def title_change(title):
    title = title.encode('ascii', 'ignore').decode('ascii')
#     title= str(title).encode('utf-8').decode("utf-8")
#     title=title.encode("ascii","ignore")
    return title

def convert_int(value):
    if value is not None:
        value = int(value)
    return value


sponsored_df1 = sponsored_df.copy()
sponsored_df1['item_price'] = sponsored_df1['item_price'].apply(convert_price)
sponsored_df1['item_sold'] = sponsored_df1['item_sold'].apply(sold_items)
sponsored_df1['title'] = sponsored_df1['title'].apply(title_change)      
sponsored_df1['seller_score'] = sponsored_df1['seller_score'].apply(convert_int)      
sponsored_df1['item_sold'] = sponsored_df1['item_sold'].apply(convert_int)   
sponsored_df1['shipping_price'] = sponsored_df1['shipping_price'].apply(convert_int)

nonsponsored_df1 = nonsponsored_df.copy()
nonsponsored_df1['item_price'] = nonsponsored_df1['item_price'].apply(convert_price)
nonsponsored_df1['item_sold'] = nonsponsored_df1['item_sold'].apply(sold_items)
nonsponsored_df1['title'] = nonsponsored_df1['title'].apply(title_change)
nonsponsored_df1['seller_score'] = nonsponsored_df1['seller_score'].apply(convert_int)   
nonsponsored_df1['item_sold'] = nonsponsored_df1['item_sold'].apply(convert_int)   
nonsponsored_df1['shipping_price'] = nonsponsored_df1['shipping_price'].apply(convert_int)


In [53]:
nonsponsored_df1[240:280]

,item_id,seller_name,seller_score,item_price,item_sold,best_offer_flag,title,returns_allowed,condition1,is_sponsored,shipping_calc,shipping_price
240,133172659775,adi_visler,1442,1159,359.0,False,Details about For Apple iPhone 11 Pro Max Tem...,True,New,No,False,0.0
241,133172659859,adi_visler,1442,945,1.0,False,Details about For iPhone 11 Pro iPhone XR XS ...,True,New,No,False,0.0
242,133172659898,adi_visler,1442,1723,6.0,False,Details about For iPhone 11 Pro Max Case Buil...,True,New,No,False,0.0
243,133172673091,adi_visler,1442,1078,5.0,False,Details about For Apple 2019 iPhone 11 Pro/iP...,True,New,No,False,0.0
244,133172685060,adi_visler,1442,1991,20.0,False,Details about For iPhone 11 Pro Max Case Clea...,True,New,No,False,0.0
245,133173208661,kika676,641,3995,2.0,False,Details about (6 Pack) For Apple iPhone 11 Pr...,True,New,No,False,0.0
246,133255940648,kika676,641,1499,NaN,False,Details about Privacy Screen Protector for Ap...,True,New,No,False,0.0
247,133259289184,kika676,641,1299,4.0,False,Details about For Apple iPhone 11 Pro Max Wal...,True,New,No,False,0.0
248,133259300882,kika676,641,1099,NaN,False,Details about For Apple iPhone 11 Pro Case Sl...,True,New,No,False,0.0
249,133266928262,kika676,641,1199,2.0,False,Details about For Apple iPhone 11 Pro Max Cas...,True,New,No,False,0.0


In [54]:
sponsored_df1.describe()

,seller_score,item_price,item_sold,shipping_price
count,63.000000,63.000000,25.000000,59.000000
mean,9652.238095,140214.698413,16.360000,2852.186441
std,17456.144038,97980.727139,59.904007,6136.200626
min,1.000000,854.000000,1.000000,0.000000
25%,189.000000,86199.500000,1.000000,0.000000
50%,3588.000000,122500.000000,2.000000,0.000000
75%,8453.000000,206599.500000,7.000000,1599.000000
max,75690.000000,510000.000000,303.000000,20233.000000


In [55]:
nonsponsored_df1.describe()

,seller_score,item_price,item_sold,shipping_price
count,9.970000e+02,997.000000,393.000000,939.000000
mean,2.130783e+04,86937.016048,51.190840,336.635783
std,1.334523e+05,92853.750785,498.833094,986.953148
min,0.000000e+00,510.000000,1.000000,0.000000
25%,3.840000e+02,2068.000000,1.000000,0.000000
50%,1.375000e+03,87900.000000,3.000000,0.000000
75%,3.258000e+03,119900.000000,7.000000,0.000000
max,2.266182e+06,749999.000000,8201.000000,13422.000000


Q2 e) Use your code script to connect to SQL (either MySQL, MariaDB, or SQLite. Do NOT use SQL GUI or command terminal). 
Create a database and name it, "eBay". 
Save the information of items in (d) into a single table named "eBay_items" (You are allowed to use only one table). This table should contain both sponsored and non-sponsored information and have a column that specifies which item is sponsored/non-sponsored. 
If an item misses ANY of the information in (d), you should insert that missing value as NULL into the table. 
Convert any price (item price and shipping price) into a "dollar-cent" format (e.g., convert $12.34 into 1234 and $12 into 1200. Make sure the two least significant digits are cents. If an item does not include cents in the price, insert zeros.) and insert the price as INT into the table.


In [56]:
#Create connection to server

conn=MySQLdb.connect(host='localhost',user='root',passwd='', charset="utf8", use_unicode=True)
cursor = conn.cursor()
cursor.execute('Create database if not exists ebay')
cursor.execute('use ebay')

C:\Users\meha\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: Warning: (1007, "Can't create database 'ebay'; database exists")
  """


0

In [57]:
## Save the dataframe for sponsored and non-sponsored in MySQL database

tableName   = "ebay_items" 

sqlEngine       = create_engine('mysql+pymysql://root:@127.0.0.1/ebay', pool_recycle=3600)
dbConnection    = sqlEngine.connect()
 
try:
     sponsored_df1.to_sql(tableName, dbConnection, if_exists='replace');
     
     nonsponsored_df1.to_sql(tableName, dbConnection, if_exists='append');
    
except ValueError as vx:
    print(vx)
except Exception as ex:   
    print(ex)
else:
    print("Table %s created successfully."%tableName);   
finally:
    dbConnection.close()

Table ebay_items created successfully.


Q2 f) Use your code script (and NOT SQL GUI or command terminal) to run summary stats on each item. Print to the screen the mean, min, max, and mean for each column, grouped by "sponsor/non-sponsor" and "condition" (group by at the same time, not separately). 
For binary categorical columns, use 0-1 conversion. For e.g., for the "returns allowed" convert YES to 1 and NO to 0 and then calculate the stats. 
If it is NOT a numerical/binary categorical column, print to the screen the count of each category level. 
You will need to ignore NULL values in your statistic calculations. 


In [58]:
def stats_calculation_count(myresult):
    '''
    This function takes the results from query and then converts 
    into a dataframe to improve the understandability of result
    
    '''
    results = pd.DataFrame(columns = ['Sponsored', 'Condition', 'Count'])

    for row in myresult:

        data = {
                 'Sponsored': row[0],
                 'Condition': row[1],
                 'Count': row[2]                
                }
        
        results = results.append(data, ignore_index = True)
        
        
    return results

In [59]:
# Calculating stats for seller_name
sql_query = '''
            select is_sponsored, condition1, count(distinct(seller_name))
            from eBay_items
            group by is_sponsored, condition1
            
            '''
conn=MySQLdb.connect(host='localhost',user='root',passwd='', charset="utf8", use_unicode=True)
cursor = conn.cursor()
cursor.execute('use ebay')
cursor.execute(sql_query)
myresult = cursor.fetchall()
cursor.close()
conn.close()
stat_row = stats_calculation_count(myresult)
print(stat_row)

   Sponsored                 Condition Count
0         No  For parts or not working    83
1         No  Manufacturer refurbished     3
2         No                       New   192
3         No                  Open box    61
4         No        Seller refurbished     8
5         No                      Used    48
6        Yes  For parts or not working     5
7        Yes  Manufacturer refurbished     1
8        Yes                       New    20
9        Yes                  Open box     8
10       Yes        Seller refurbished     2
11       Yes                      Used     2


In [60]:
# Calculating stats for title
sql_query = '''
            select is_sponsored, condition1, count(distinct(title))
            from eBay_items
            group by is_sponsored, condition1
            
            '''
conn=MySQLdb.connect(host='localhost',user='root',passwd='', charset="utf8", use_unicode=True)
cursor = conn.cursor()
cursor.execute('use ebay')
cursor.execute(sql_query)
myresult = cursor.fetchall()
cursor.close()
stat_row = stats_calculation_count(myresult)
print(stat_row)

   Sponsored                 Condition Count
0         No  For parts or not working   107
1         No  Manufacturer refurbished     3
2         No                       New   573
3         No                  Open box   105
4         No        Seller refurbished    45
5         No                      Used    68
6        Yes  For parts or not working     6
7        Yes  Manufacturer refurbished     1
8        Yes                       New    39
9        Yes                  Open box    11
10       Yes        Seller refurbished     3
11       Yes                      Used     3


In [61]:
def stats_calculation(myresult):
    '''
    This function takes the results from query and then converts 
    into a dataframe to improve the understandability of result
    
    '''
    results = pd.DataFrame(columns = ['Sponsored', 'Condition', 'Min', 'Max', 'Standard_Dev'])
    for row in myresult:
        
        data = {
                 'Sponsored': row[0],
                 'Condition': row[1],
                 'Min': row[2],
                 'Max': row[3],
                 'Standard_Dev': row[4],
                
                }
        results = results.append(data, ignore_index = True)
        
    return results
    

In [62]:
# Calculating stats for seller_score
sql_query = '''
            select is_sponsored, condition1, min(seller_score), max(seller_score), std(seller_score)
            from eBay_items
            where seller_score is not null
            group by is_sponsored, condition1
            '''
conn=MySQLdb.connect(host='localhost',user='root',passwd='', charset="utf8", use_unicode=True)
cursor = conn.cursor()
cursor.execute('use ebay')
cursor.execute(sql_query)
myresult = cursor.fetchall()
cursor.close()
stat_row = stats_calculation(myresult)
print(stat_row)

   Sponsored                 Condition   Min      Max  Standard_Dev
0         No  For parts or not working     0    46075  9.815516e+03
1         No  Manufacturer refurbished  1512  2266182  1.015441e+06
2         No                       New     0   259002  1.254886e+04
3         No                  Open box     0   326043  5.448270e+04
4         No        Seller refurbished    68  2266179  4.439078e+05
5         No                      Used     0   259002  6.007049e+04
6        Yes  For parts or not working   127    75690  2.776198e+04
7        Yes  Manufacturer refurbished   299      299  0.000000e+00
8        Yes                       New     4    60477  1.012009e+04
9        Yes                  Open box     1    70744  2.281389e+04
10       Yes        Seller refurbished   299    50793  2.380310e+04
11       Yes                      Used   256     3588  1.570720e+03


In [63]:
# Calculating stats for item_price
sql_query = '''
            select is_sponsored, condition1, min(item_price), max(item_price), std(item_price)
            from eBay_items
            where item_price is not null
            group by is_sponsored, condition1
            '''
conn=MySQLdb.connect(host='localhost',user='root',passwd='', charset="utf8", use_unicode=True)
cursor = conn.cursor()
cursor.execute('use ebay')
cursor.execute(sql_query)
myresult = cursor.fetchall()
cursor.close()
stat_row = stats_calculation(myresult)
print(stat_row)

   Sponsored                 Condition     Min     Max   Standard_Dev
0         No  For parts or not working   30000  119900   15142.986818
1         No  Manufacturer refurbished   84999   91995    2942.561847
2         No                       New     510  749999  112279.440305
3         No                  Open box     600  214900   26462.539937
4         No        Seller refurbished   78900  137600   11696.831606
5         No                      Used   74777  229900   24891.545052
6        Yes  For parts or not working   69995  115000   15039.019724
7        Yes  Manufacturer refurbished  104000  104000       0.000000
8        Yes                       New     854  510000  115576.612708
9        Yes                  Open box   87499  229900   47726.461208
10       Yes        Seller refurbished   84900  117500   14335.348696
11       Yes                      Used   79900  109900   14142.135624


In [64]:
# Calculating stats for item_sold
sql_query = '''
            select is_sponsored, condition1, min(item_sold), max(item_sold), stddev(item_sold)
            from eBay_items
            where item_sold is not null
            group by is_sponsored, condition1
            '''
conn=MySQLdb.connect(host='localhost',user='root',passwd='', charset="utf8", use_unicode=True)
cursor = conn.cursor()
cursor.execute('use ebay')
cursor.execute(sql_query)
myresult = cursor.fetchall()
cursor.close()
stat_row = stats_calculation(myresult)
print(stat_row)

  Sponsored                 Condition    Min     Max  Standard_Dev
0        No  For parts or not working    1.0    11.0      3.826225
1        No  Manufacturer refurbished  113.0   113.0      0.000000
2        No                       New    1.0  8201.0    529.119531
3        No                  Open box    1.0    35.0      8.854511
4        No        Seller refurbished    1.0    30.0      9.617692
5        No                      Used    1.0     3.0      0.640312
6       Yes  For parts or not working    1.0     1.0      0.000000
7       Yes                       New    1.0    18.0      5.061352
8       Yes                  Open box    1.0   303.0    129.926903
9       Yes        Seller refurbished    1.0     9.0      4.000000


In [65]:
# Calculating stats for return_allowed

sql_query = '''
            select is_sponsored, condition1, min(returns_allowed), max(returns_allowed), stddev(returns_allowed)
            from eBay_items
            where returns_allowed is not null
            group by is_sponsored, condition1
            '''
conn=MySQLdb.connect(host='localhost',user='root',passwd='', charset="utf8", use_unicode=True)
cursor = conn.cursor()
cursor.execute('use ebay')
cursor.execute(sql_query)
myresult = cursor.fetchall()
cursor.close()
stat_row = stats_calculation(myresult)
print(stat_row)

   Sponsored                 Condition Min Max  Standard_Dev
0         No  For parts or not working   0   1      0.472465
1         No  Manufacturer refurbished   0   1      0.471405
2         No                       New   0   1      0.441578
3         No                  Open box   0   1      0.488312
4         No        Seller refurbished   0   1      0.249444
5         No                      Used   0   1      0.484587
6        Yes  For parts or not working   0   1      0.471405
7        Yes  Manufacturer refurbished   1   1      0.000000
8        Yes                       New   0   1      0.486504
9        Yes                  Open box   0   1      0.497930
10       Yes        Seller refurbished   1   1      0.000000
11       Yes                      Used   0   1      0.471405


In [66]:
# Calculating stats for shipping

sql_query = '''
            select is_sponsored, condition1, min(shipping_price), max(shipping_price), stddev(shipping_price)
            from eBay_items
            where shipping_price is not null
            group by is_sponsored, condition1
            '''
conn=MySQLdb.connect(host='localhost',user='root',passwd='', charset="utf8", use_unicode=True)
cursor = conn.cursor()
cursor.execute('use ebay')
cursor.execute(sql_query)
myresult = cursor.fetchall()
cursor.close()
stat_row = stats_calculation(myresult)
print(stat_row)

   Sponsored                 Condition  Min      Max  Standard_Dev
0         No  For parts or not working  0.0   2200.0    523.544526
1         No  Manufacturer refurbished  0.0   1575.0    742.462120
2         No                       New  0.0   8437.0   1019.142780
3         No                  Open box  0.0   2200.0    487.711204
4         No        Seller refurbished  0.0    949.0    233.878355
5         No                      Used  0.0  13422.0   1750.824844
6        Yes  For parts or not working  0.0    700.0    260.874597
7        Yes  Manufacturer refurbished  0.0      0.0      0.000000
8        Yes                       New  0.0  20233.0   7216.317689
9        Yes                  Open box  0.0      0.0      0.000000
10       Yes        Seller refurbished  0.0      0.0      0.000000
11       Yes                      Used  0.0   1500.0    707.106781


In [67]:
# Calculating stats for make_offer

sql_query = '''
            select is_sponsored, condition1, min(best_offer_flag), max(best_offer_flag), stddev(best_offer_flag)
            from eBay_items
            where best_offer_flag is not null
            group by is_sponsored, condition1
            '''
conn=MySQLdb.connect(host='localhost',user='root',passwd='', charset="utf8", use_unicode=True)
cursor = conn.cursor()
cursor.execute('use ebay')
cursor.execute(sql_query)
myresult = cursor.fetchall()
cursor.close()
stat_row = stats_calculation(myresult)
print(stat_row)

   Sponsored                 Condition Min Max  Standard_Dev
0         No  For parts or not working   0   1      0.458258
1         No  Manufacturer refurbished   0   0      0.000000
2         No                       New   0   1      0.297368
3         No                  Open box   0   1      0.472494
4         No        Seller refurbished   0   0      0.000000
5         No                      Used   0   1      0.336781
6        Yes  For parts or not working   0   1      0.372678
7        Yes  Manufacturer refurbished   0   0      0.000000
8        Yes                       New   0   1      0.449998
9        Yes                  Open box   0   1      0.481046
10       Yes        Seller refurbished   0   0      0.000000
11       Yes                      Used   0   1      0.471405


Q2 g) Use the stats in (f) and tell in your own words, how sponsored and non-sponsored items appear to be different. Is there a variable/column that can be used to predict the sponsor/non-sponsor items. (You do NOT need to run any model or statistic calculations for this part. Make your judgment only by looking at the stats you printed in (f))


1.Returns allowed is more common in non-sponsored than sponsored items

2.About 40% items avaliable in the sponsored have an option of make an offer 
  while 70% items have an option of make an offer in non-sponsored items

3.The avg seller score for non-sponsored item is higher than the 
  avg seller score for sponsored item

4.The avg item price for sponsored is higher than 
  the avg item price of a non-sponsored item

5.Number of distinct title and seller in non-sponsored is more than sponsored

The above variables like make offer, seller score, item price, returns allowed, distinct seller name and title
can be used to predict whether the item is sponsored or non-sponsored

